## Step 1: Invoke Bedrock Models to get _Inferences_ on a user provided Dataset
---

This notebook does as follows:

1. Generates inferences on a user provided dataset, using Foundation models on Amazon Bedrock

1. Uses [Litellm](https://www.litellm.ai/) as an interface to interact with the Bedrock API

1. Uses `Ray`, which is used to run inferences in an asynchronous manner

1. Records metrics like the `p90, p95` latency, `prompt token counts`, `completion token counts`, and more.

1. Saves all the combined _model responses_ to user questions from the source dataset in a `all_results.csv` that is used later in the _evaluation step_ for the evaluation process.

In [ ]:
# import the libraries
import os
import ray
import json
import yaml
import time
import boto3
import logging
import pandas as pd
from pathlib import Path
from functools import reduce
from litellm import completion
from typing import Dict, List, Optional

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# initialize the ray service to run async calls in parallel to bedrock easily
if ray.is_initialized():
    ray.shutdown()
ray.init()

In [ ]:
# global constants
CONFIG_FILE_PATH = "config.yaml"

# read the config yaml file
fpath = CONFIG_FILE_PATH
with open(fpath, 'r') as yaml_in:
    config = yaml.safe_load(yaml_in)
logger.info(f"config read from {fpath} -> {json.dumps(config, indent=2)}")

In [ ]:
# initialize all global variables that are used across this notebook hydrated from the `config.yaml` file

# name of your source xlsx/xls/csv file 
FILE_NAME: str = config['dir_info']['dataset_file_name']
# data directory
DATA_DIR: str = config['dir_info']['data_dir']
FILE_RELATIVE_PATH: str = os.path.join(config['dir_info']['dataset_dir'], FILE_NAME)
INPUT_FPATH: str = os.path.join(DATA_DIR, FILE_RELATIVE_PATH)
USER_PROMPT_COL: str = config['dataset_info']['user_question_col']
SYSTEM_PROMPT_COL: str = config['dataset_info']['system_prompt_col']
INFERENCE_PARAMETERS: Dict = config['inference_parameters']
ON_LIST = list(filter(None, [USER_PROMPT_COL, 
                            SYSTEM_PROMPT_COL]))
TEXT_WRAP_WIDTH_SIZE: int = 80

# result files
ALL_RESULTS_FPATH = os.path.join(DATA_DIR, config['dir_info']['metrics'])
INFERENCE_LATENCY_SUMMARY_FPATH = os.path.join(ALL_RESULTS_FPATH, config['dir_info']['inference_latency_summary_fname'])
bedrock_model_ids: List[str] = config['bedrock_fms_to_test']

### Get Inference for the given dataset
---

This portion of the notebook gets inference using `Ray` (which is used to handle asynchronous calls to `Litellm`) to get inferences from the user questions in the given dataset

In [ ]:
def generate_task_inference(model_id: str, 
                            user_prompt: str, 
                            system_prompt: str, 
                            inference_parameters: Dict) -> Dict:
    """
    This function takes in a dictionary (which contains information on the user data and prompts) 
    to generate inference using a bedrock model id, and returns a dictionary containing the model 
    completion, and latency (in seconds).
    """
    # represents the service name
    service_name: str = "bedrock"
    temperature = inference_parameters.get('temperature', 0.1)
    caching = inference_parameters.get('caching', False)
    max_tokens = inference_parameters.get("max_tokens", 500)
    # represents creating the bedrock model to invoke the litellm api for response for titan, llama and claude
    bedrock_model: str = f"{service_name}/{model_id}"
    # represents the current aws region
    aws_region = boto3.Session().region_name
    # initialize the response dict
    ret = dict(user_prompt=user_prompt,
               system_prompt=system_prompt,
               completion=None,
               model_id=model_id,
               time_taken_in_seconds=None,
               prompt_token_count=None,
               completion_token_count=None,
               exception=None)
    # custom messages formatting for when the user/system roles are given together
    messages=[{ "content": user_prompt, "role": "user"}]
    if system_prompt is not None:
        messages.append({"content": system_prompt, "role": "system"})

    # set the env var for aws_region
    os.environ["AWS_REGION_NAME"] = aws_region 
    try:
        print(f"Invoking {bedrock_model}......")
        response = completion(model=bedrock_model,
                              messages=messages,
                              temperature=temperature,
                              max_tokens=max_tokens,
                              caching=caching)

        # iterate through the entire model response
        for idx, choice in enumerate(response.choices):
            print(f"choice {idx+1} of {len(response.choices)} ")
            # extract the message and the message's content from litellm
            if choice.message and choice.message.content:
                # extract the response from the dict
                ret["completion"] = choice.message.content.strip()

        # Commenting out the code below that records the number of input and output tokens.
        # Extract number of input and completion prompt tokens (this is the same structure for embeddings and text generation models on Amazon Bedrock)
        ret['prompt_token_count'] = response.usage.prompt_tokens
        ret['completion_token_count'] = response.usage.completion_tokens
        # Extract latency in seconds
        latency_ms = response._response_ms
        ret['time_taken_in_seconds']  = latency_ms / 1000
    except Exception as e:
        print(f"Exception occurred during invoking {model_id}, exception={e}")
        ret['exception'] = e
    return ret

In [ ]:
@ray.remote
def async_generate_task_inference(input: Dict, model_id: str, inference_parameters: Dict) -> Dict:
    resp = generate_task_inference(model_id, input.get('user_prompt'), input.get('system_prompt'), inference_parameters)
    resp_this_model = {"model_id": model_id,
                       USER_PROMPT_COL: input.get('user_prompt'),
                       SYSTEM_PROMPT_COL: input.get('system_prompt'),
                       f"{model_id}-response": resp['completion'],
                       f"{model_id}-time_taken_in_seconds": resp['time_taken_in_seconds'],
                       f"{model_id}-prompt_token_count": resp['prompt_token_count'],
                       f"{model_id}-completion_token_count": resp['completion_token_count'],
                       f"{model_id}-exception": resp['exception']}
    return resp_this_model

In [ ]:
logger.info(f"File name to be processed: {INPUT_FPATH}")
data_file = Path(INPUT_FPATH)
if data_file.suffix == '.csv':
    logger.info(f"processing the csv file: {data_file}")
    original_eval_df = pd.read_csv(data_file)
elif data_file.suffix in ['.xls', '.xlsx']:
    logger.info(f"processing the xls/xlsx file: {data_file}")
    original_eval_df = pd.read_excel(data_file)
else:
    raise ValueError(f"Unsupported file format: {data_file.suffix}")
logger.info(f"input data frame shape is {original_eval_df.shape}")
# drop the columns that have all 'NaN' values
original_eval_df = original_eval_df.dropna(axis=1, how='all')
original_eval_df.head(10)

In [ ]:
original_eval_list = json.loads(original_eval_df.to_json(orient='records'))
original_eval_list

In [ ]:
# list of the bedrock model ids that are used in generating inferences
bedrock_model_ids: List[str] =[d['model_id'] for d in config['bedrock_fms_to_test']]
bedrock_model_ids

### Run the inferences to get model responses in parallel using `Ray`

In [ ]:
erroneous_count: int = 0
resp_list = []
n: int = config.get('parallel_inference_count')
st_overall = time.perf_counter()
# Iterate over each bedrock model ID
for model_id in bedrock_model_ids:
    logger.info(f"going to get inference from model={model_id}")
    list_of_lists = [original_eval_list[i * n:(i + 1) * n] for i in range((len(original_eval_list) + n - 1) // n )]
    st = time.perf_counter()
    for idx, sublist in enumerate(list_of_lists):
        logger.info(f"processing sublist={idx+1}/{len(list_of_lists)} for model_id={model_id}")
        for input in sublist:
            print(f"input logged: {input}")
            try:
                input_dict = dict(user_prompt=input.get(USER_PROMPT_COL), system_prompt=input.get(SYSTEM_PROMPT_COL))
                result = ray.get(async_generate_task_inference.remote(input_dict, model_id, INFERENCE_PARAMETERS))
                resp_list.append(result)
            except Exception as e:
                logger.error(f"Error processing input: {input} for model_id={model_id}, error: {e}")
                erroneous_count += 1

    elapsed_time = time.perf_counter() - st
    logger.info(f"total time taken for {len(original_eval_list)} with model={model_id} is {elapsed_time:0.2f}")
elapsed_time = time.perf_counter() - st_overall
logger.info(f"total time taken for {len(original_eval_list)} with models={bedrock_model_ids} is {elapsed_time:0.2f}")
logger.info(f"total erroneous count: {erroneous_count}")

In [ ]:
# view some responses generated
resp_list[:5]

In [ ]:
df_list = []
for model_id in bedrock_model_ids:
    df_list.append(pd.DataFrame([r for r in resp_list if r['model_id'] == model_id]).drop(['model_id'], axis=1))    
from functools import reduce
logger.info(f"on_list: {ON_LIST}")
try:
    # if the system prompt is separately provided, merge on that column too else, just use the user
    # column for the merge
    df_resp = reduce(lambda x, y: pd.merge(x, y, on=ON_LIST), 
                    df_list)
except Exception as e:
    logger.error(f"df was not merged: {e}")
logger.info(f"shape of response data frame={df_resp.shape}")

In [ ]:
# view the data frame
df_resp = df_resp.loc[:, ~df_resp.columns.str.startswith('None')]
df_resp.head(10)

In [ ]:
# get the original/target responses if any and merge it with the current df
try: 
    if df_resp is not None and config['dataset_info']['pre_existing_response_col'] is not None:
        df_resp_all = pd.merge(left=df_resp, right=original_eval_df, how="left",
                            left_on=ON_LIST, 
                            right_on=ON_LIST)
except Exception as e:
    logger.error(f"Could not perform the merge with the original data frame: {e}")


In [ ]:
# view the current data in the df
df_resp_all.head(10)

### Record the `p50` and `p95` inference latencies in a `txt` file

In [ ]:
time_taken_in_seconds_cols = [c for c in df_resp_all.columns if 'time_taken_in_seconds' in c]
Latency_cols = [c for c in df_resp_all.columns if 'Latency ' in c]
all_latency_cols_of_interest = time_taken_in_seconds_cols + Latency_cols
summary = ""
for c in all_latency_cols_of_interest:
    quantiles = list(round(df_resp_all[c].quantile([0.5, 0.95]), 2))
    s = f"[p50, p95] for {c}={quantiles}\n"
    summary += s
    logger.info(s)
Path(INFERENCE_LATENCY_SUMMARY_FPATH).write_text(summary)

### Upload the overall results to a `results.csv` file

In [ ]:
cols = list(df_resp_all.columns)
user_input_index = df_resp_all.columns.get_loc(config['dataset_info']['user_question_col'])
response_cols = [col for col in df_resp_all.columns if col.endswith('-response')]
for col in response_cols:
    cols.pop(cols.index(col))
# Reinsert the response columns right after the user_input column
for col in reversed(response_cols):
    cols.insert(user_input_index + 1, col)
df_resp_all = df_resp_all[cols]

# Save the DataFrame to a CSV file
os.makedirs(ALL_RESULTS_FPATH, exist_ok=True)
all_results_csv_fpath: str = os.path.join(ALL_RESULTS_FPATH, 
                                          config['dir_info']['all_results_file_name'])
df_resp_all.to_csv(all_results_csv_fpath, index=False)
df_resp_all.head(10)